In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.adv.guendel19 import densenet
from src.metrics.metrics import F2Score
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

Using TensorFlow backend.


In [3]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
!remote_access/get_tunnels.sh

URLs open for the following services:
[('tensorboard', 'http://d39e353b12ea.ngrok.io'), ('jupyterlab', 'http://39fdc6865faa.ngrok.io'), ('jupyternotebook', 'http://16e70049dd92.ngrok.io')]


In [5]:
metrics = [tf.keras.metrics.AUC(multi_label=True, name="auc"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
           F2Score(name="f2_score"),
           tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")
          ]

In [6]:

chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  "Chexpert small batches",
                                  epochs=10,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=32,
                                  metrics = metrics)

chexpert_benchmark.loss = WeightedBinaryCrossentropy(*compute_class_weight(chexpert_benchmark.traingen))  
chexpert_benchmark.summary()

"The benchmark was initialized for the chexpert_full dataset with batch size of 32, shuffel set to True and images rescaled to dimension (256, 256).\nThe training was done for 10 epochs using the Adam optimizer and weighted_binary_crossentropy loss.\nA total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.\nThe traing set included 143328 number of sample, the validation set 34726, and the test set 45360. "

In [7]:
chestxray14_columns = ['Edema', 'Atelectasis', 'Pneumonia',
                       'Pleural_Thickening', 'Cardiomegaly', 'Infiltration', 'Consolidation',
                       'Fibrosis', 'No Finding', 'Effusion', 'Nodule', 'Mass', 'Hernia',
                       'Emphysema', 'Pneumothorax']

chestxray14_benchmark = Benchmark(Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY")),
                                  chestxray14_columns,
                                  "Chestxray 14 NIH weighted",
                                  epochs=10,
                                  train_labels="meta/data/labels.csv",
                                  path_column="Image Index",
                                  path_column_prefix="images/",
                                  split_group='Patient ID',
                                  batch_size=16,
                                  metrics=metrics)
chestxray14_benchmark.loss = WeightedBinaryCrossentropy(*compute_class_weight(chestxray14_benchmark.traingen))  
chestxray14_benchmark.as_dict()

{'benchmark_name': 'Chestxray 14 NIH weighted',
 'dataset_name': 'chestxray14_scale_256',
 'dataset_folder': 'data/chestxray14/scale_256',
 'models_dir': 'models',
 'epochs': 10,
 'optimizer': 'Adam',
 'loss': <src.metrics.losses.WeightedBinaryCrossentropy at 0x7fcf2c0524e0>,
 'metrics': ['AUC', 'Precision', 'Recall', 'F2Score', 'BinaryAccuracy'],
 'label_columns': ['Edema',
  'Atelectasis',
  'Pneumonia',
  'Pleural_Thickening',
  'Cardiomegaly',
  'Infiltration',
  'Consolidation',
  'Fibrosis',
  'No Finding',
  'Effusion',
  'Nodule',
  'Mass',
  'Hernia',
  'Emphysema',
  'Pneumothorax'],
 'path_column': 'Image Index',
 'path_column_prefix': 'images/',
 'shuffle': True,
 'batch_size': 16,
 'dim': (256, 256),
 'n_channels': 3,
 'nan_replacement': 0,
 'unc_value': -1,
 'u_enc': 'uzeroes',
 'drop_last': True,
 'train_num_samples': 71021,
 'valid_num_samples': 18231,
 'test_num_samples': 22868}

In [8]:
model_resnet_chestxray14 = SimpleBaseArchitecture(ResNet152V2, len(chestxray14_columns))
model_resnet_chexpert = SimpleBaseArchitecture(ResNet152V2, len(chexpert_columns))

model_densenet_chestxray14 = SimpleBaseArchitecture(DenseNet121, len(chestxray14_columns))
model_densenet_chexpert = SimpleBaseArchitecture(DenseNet121, len(chexpert_columns))

model_inceptionnet_chestxray14 = SimpleBaseArchitecture(InceptionV3, len(chestxray14_columns))
model_inceptionnet_chexpert = SimpleBaseArchitecture(InceptionV3, len(chexpert_columns))

In [9]:
experiment_resnet_chexpert = Experiment(chexpert_benchmark, model_resnet_chexpert, "ResNet152V2_CheXpert_weighted_32")
experiment_densenet_chexpert = Experiment(chexpert_benchmark, model_densenet_chexpert, "DenseNet121_CheXpert_weighted_32")
experiment_inceptionnet_chexpert = Experiment(chexpert_benchmark, model_inceptionnet_chexpert, "InceptionV3_CheXpert_weighted_32")

In [10]:
experiment_resnet_chestxray14 = Experiment(chestxray14_benchmark, model_resnet_chestxray14, "ResNet152V2_ChestXray14_weighted_16")
experiment_densenet_chestxray14 = Experiment(chestxray14_benchmark, model_densenet_chestxray14, "DenseNet121_ChestXray14_weighted_16")
experiment_inceptionnet_chestxray14 = Experiment(chestxray14_benchmark, model_inceptionnet_chestxray14, "InceptionV3_ChestXray14_weighted_16")

In [11]:
print(experiment_resnet_chexpert.model_description)

Trained ResNet152V2_CheXpert_weighted_32 architecture using the 'Chexpert small batches' benchmark. The benchmark was initialized for the chexpert_full dataset with batch size of 32, shuffel set to True and images rescaled to dimension (256, 256).
The training was done for 10 epochs using the Adam optimizer and weighted_binary_crossentropy loss.
A total of 14 labels/pathologies were included in the training and encoded using the 'uzeroes' method.
The traing set included 143328 number of sample, the validation set 34726, and the test set 45360. 


In [ ]:
experiment_resnet_chexpert_result =  experiment_resnet_chexpert.run()

Epoch 1/10
   1/4479 [..............................] - ETA: 0s - loss: 1.6485 - auc: 0.4364 - precision: 0.1245 - recall: 0.4627 - f2_score: 0.2998 - binary_accuracy: 0.4330WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
 828/4479 [====>.........................] - ETA: 10:47 - loss: 1.3036 - auc: 0.6569 - precision: 0.2475 - recall: 0.6763 - f2_score: 0.5023 - binary_accuracy: 0.6071

In [ ]:
experiment_densenet_chexpert_result =  experiment_densenet_chexpert.run()

In [ ]:
experiment_inceptionnet_chexpert_result =  experiment_inceptionnet_chexpert.run()

In [ ]:
experiment_resnet_chestxray14_result =  experiment_resnet_chestxray14.run()

In [ ]:
experiment_densenet_chestxray14_result =  experiment_densenet_chestxray14.run()

In [ ]:
experiment_inceptionnet_chestxray14_result =  experiment_inceptionnet_chestxray14.run()

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
model_guendel_chestxray14 = densenet(classes=len(chestxray14_columns))
model_guendel_chexpert = densenet(classes=len(chexpert_columns))

experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14, "Guendel19_ChestXray14")
experiment_guendel_chexpert = Experiment(chexpert_benchmark, model_guendel_chexpert, "Guendel19_CheXpert")



In [ ]:
experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()